In [ ]:
import pandas as pd
import numpy as np
import os
import re
import string
from collections import Counter, defaultdict
from rapidfuzz import process, fuzz
import dask.dataframe as dd
import sys

df = pd.read_csv("Filtered_Ordered_Amended_Correct_No_IR.csv")
df = df.rename({"Keyword": "Keywords", "Para": "Paragraph"}, axis = 1)
columns = ["Keywords", "Paragraph", "Date", "Title", "Subtitle", "Report"]
edf = pd.read_excel("entryfiles_combined_v5_withparagraphs.xlsx")

In [9]:
compustat_df = pd.read_csv("ciqcompany_mergedwithgvkeyandcountry.csv")
compustat_df = compustat_df[compustat_df["gvkey"].notnull()]
compustat_df

,companyid,companyname,countryid,gvkey,country,isocountry2
5,1.850700e+04,2M Invest A/S,55.0,235716.0,Denmark,DK
7,1.851100e+04,3i Group plc,212.0,210835.0,United Kingdom,GB
14,1.852700e+04,ABB Ltd,195.0,210418.0,Switzerland,CH
70,1.867100e+04,Albemarle Corporation,213.0,29751.0,United States,US
86,1.871100e+04,The Allstate Corporation,213.0,28349.0,United States,US
...,...,...,...,...,...,...
25531497,1.679592e+09,PT. Geoprima Solusi Tbk,94.0,350207.0,Indonesia,ID
25531557,1.679596e+09,"Guotai Asset Management Co., Ltd. - Cathay Pac...",43.0,350228.0,China,CN
25535832,1.679868e+09,PT Prima Andalan Mandiri Tbk,94.0,350221.0,Indonesia,ID
25539907,1.680025e+09,Elementia Materiales S.A.B. De C.V.,131.0,350235.0,Mexico,MX


In [6]:
new_df = df[columns].rename({"final_gvkey": "gvkey", "final_country": "country"}, axis = 1)
new_df

,Keywords,Paragraph,Date,Title,Subtitle,Report
0,hurdle rate,We expect to reach chief a return on capital i...,2010-10-28,ACE LTD - PRELIMINARY TRANSCRIPT,ACE - Event Transcript of ACE Limited conferen...,16811609
1,hurdle rate,Our first priority for our free cash flow is t...,2005-02-17,ADVANCE AUTO PARTS INC,,10135055
2,hurdle rate,Danielle Fox - Merrill Lynch - Analyst My next...,2005-02-17,ADVANCE AUTO PARTS INC,,10135055
3,hurdle rate,Jos Streppel - AEGON N.V. - CFO Yes. It's a bi...,2007-03-08,AEGON NV,"AEG conference call Final transcript, 8-Mar-07...",12061675
4,hurdle rate,Go to the next slide. We had sales growth in o...,2007-08-09,AEGON NV - FINAL TRANSCRIPT,"AEG conference call Final transcript, 9-Aug-07...",12520697
...,...,...,...,...,...,...
30813,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059
30814,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045
30815,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248
30816,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619


In [7]:
df = pd.read_csv("Filtered_Ordered_Amended_Correct_No_IR.csv")
df = df.rename({"Keyword": "Keywords", "Para": "Paragraph"}, axis = 1)
columns = ["Keywords", "Paragraph", "Date", "Title", "Subtitle", "Report"]
edf = pd.read_excel("entryfiles_combined_v5_withparagraphs_andgvkey.xlsx")
edf

,Keywords,Paragraph,Date,gvkey,Title,Subtitle,Report
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,24040.0,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,65570.0,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,122096.0,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,6799.0,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965
4,irr,people per year. It has 300 retail stores inc...,NaN,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,8710501
...,...,...,...,...,...,...,...
80637,cost of equity,We're confident Stage 3 is well positioned for...,2021-09-07,29453.0,CHENIERE ENERGY INC,LNG.A - Event Transcript of Cheniere Energy In...,72918196
80638,cost of debt,Michael Webber - Webber Research & Advisory LL...,2021-09-07,29453.0,CHENIERE ENERGY INC,LNG.A - Event Brief of Cheniere Energy Inc con...,72918224
80639,irr,"On ESG, we just discussed. On performance, oka...",2021-09-07,296577.0,APERAM SA,APAM.AS - Event Transcript of Aperam SA confer...,72931747
80640,interest rate,Looking at Global Broking revenue by asset cla...,2021-09-07,241016.0,TP ICAP GROUP PLC,TCAPI.L - Event Transcript of TP ICAP Group PL...,72939535


In [10]:
edf_merge = edf.merge(compustat_df.drop_duplicates("gvkey"), on = "gvkey", how = "left")
edf_merge = edf_merge[["Keywords", "Paragraph", "Date", "Title", "Subtitle", "Report", "gvkey", "companyname", "country"]]
edf_merge

,Keywords,Paragraph,Date,Title,Subtitle,Report,gvkey,companyname,country
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,24040.0,"Ionis Pharmaceuticals, Inc.",United States
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,65570.0,American Italian Pasta Company,United States
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,122096.0,"Insight Communications Company, Inc.",United States
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,6799.0,KeySpan Corporation,United States
4,irr,people per year. It has 300 retail stores inc...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,8710501,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
80637,cost of equity,We're confident Stage 3 is well positioned for...,2021-09-07,CHENIERE ENERGY INC,LNG.A - Event Transcript of Cheniere Energy In...,72918196,29453.0,"Cheniere Energy, Inc.",United States
80638,cost of debt,Michael Webber - Webber Research & Advisory LL...,2021-09-07,CHENIERE ENERGY INC,LNG.A - Event Brief of Cheniere Energy Inc con...,72918224,29453.0,"Cheniere Energy, Inc.",United States
80639,irr,"On ESG, we just discussed. On performance, oka...",2021-09-07,APERAM SA,APAM.AS - Event Transcript of Aperam SA confer...,72931747,296577.0,Aperam S.A.,Luxembourg
80640,interest rate,Looking at Global Broking revenue by asset cla...,2021-09-07,TP ICAP GROUP PLC,TCAPI.L - Event Transcript of TP ICAP Group PL...,72939535,241016.0,Tullett Prebon Group Holdings plc,United Kingdom


In [11]:
edf_merge.to_excel("old_conf_call_compustat_match.xlsx")

In [13]:
combined = pd.concat([edf_merge, new_df])
combined

,Keywords,Paragraph,Date,Title,Subtitle,Report,gvkey,companyname,country
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,24040.0,"Ionis Pharmaceuticals, Inc.",United States
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,65570.0,American Italian Pasta Company,United States
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,122096.0,"Insight Communications Company, Inc.",United States
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,6799.0,KeySpan Corporation,United States
4,irr,people per year. It has 300 retail stores inc...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,8710501,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
30813,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,NaN,NaN,NaN
30814,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,NaN,NaN,NaN
30815,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,NaN,NaN,NaN
30816,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,NaN,NaN,NaN


In [14]:
combined.to_excel("full_conf_calls_compustat_match.xlsx")

### Fuzzy Matching

In [15]:
compustat_df = pd.read_csv("ciqcompany_mergedwithgvkeyandcountry.csv")
compustat_df = compustat_df[compustat_df["gvkey"].notnull()]

old_words = ["earnings conference call","conference call on productivity", "earnings release conference", "financial release conference",
        "conference call regarding", "earnings conference", "comprehensive review", "final transcript", "edited transcript",
        "week conference", "conference call", "edited brief", "preliminary brief", "earnings call", "earning call",
        "preliminary transcript", "final transcript", "call","cal","merger","c", "earning","earnings", "to discuss",
        "group","plc","ltd","limited","ag","corp","corporation","Incorporation","laboratories","labs","the","proposed","propose",
        "holdings","oyj","inc","conference","co", "final","preliminary","and","&",
        "company","trust","investment","investments","sln","sa","s.p.a.","spa","transc", "quarter","st","nd","rd","th",
        "q", "jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
to_remove = ["cal", "c", "and", "&", "st", "nd", "rd", "th", "q"]
new_words = list(set(old_words) - set(to_remove))
new_words = [i.upper() for i in new_words]

In [16]:
def get_relevant_info(title):
    if " - " in title:
        l = title.split(" - ")
        name_info = l[0]
        ticker_info = name_info.split()[-1].replace("(", "").replace(")", "")
        other_info = l[1]
        return name_info, ticker_info, other_info
    else:
        return title, np.nan, np.nan
    
def tickerBrackets(name, t):
    t_string = "({})".format(t)
    try:
        ind = name.index(t_string)
        cleansed_name = name[:(ind - 1)]
        return cleansed_name
    except:
        return name
    
def remove_punc(words, l):
    no_punc_words = [s.translate(str.maketrans("", "", string.punctuation)) for s in words]
    return " ".join(no_punc_words)

def remove_punctuations_and_words(words, l):
    no_punc_words = [s.translate(str.maketrans("", "", string.punctuation)) for s in words]
    filtered = [i for i in no_punc_words if i not in l]
    return " ".join(filtered)
    
def get_names(df, compustat_df, new_words):
    names = []
    tickers = []
    miscs = []
    for title in df["Title"]:
        name, ticker, misc = get_relevant_info(title)
        names.append(name)
        tickers.append(ticker)
        miscs.append(misc)
    
    df["Name"], df["Ticker"], df["Other Information"] = names, tickers, miscs
    df["Cleaned_Name"] = df.apply(lambda x: tickerBrackets(x["Name"], x["Ticker"]), axis = 1)
    df["Cleaned_Name_No_Punctuations"] = df["Cleaned_Name"].str.split().apply(lambda x: remove_punc(x, new_words))
    df["Cleaned_Name_No_Punctuations_And_Removed_Words"] = df["Cleaned_Name"].str.split().apply(lambda x: remove_punctuations_and_words(x, new_words))
    
    compustat_df["companyname_No_Punctuations"] = compustat_df["companyname"].str.upper().str.split().apply(lambda x: remove_punc(x, new_words))
    compustat_df["companyname_No_Punctuations_And_Removed_Words"] = compustat_df["companyname"].str.upper().str.split().apply(lambda x: remove_punctuations_and_words(x, new_words))
    return df, compustat_df

In [22]:
#full_df = pd.read_excel("updated_full_conf_calls_match.xlsx")
#full_df
df[df["Title"] == "nan"]

,Keywords,Paragraph,Date,Title,Subtitle,Report,gvkey,companyname,country,Compustat_Closest_Match,Compustat_Closest_Match_Similarity,Compustat_Closest_Match_Fullname,Compustat_Closest_Match_gvkey,Compustat_Closest_Match_Country,Hassan_Closest_Match,Hassan_Closest_Match_Similarity,Hassan_Closest_Match_Fullname,Hassan_Closest_Match_gvkey,Hassan_Closest_Match_Country
4,irr,people per year. It has 300 retail stores inc...,NaN,nan,NaN,8710501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,return on assets,Accounts receivable and other current assets w...,NaN,nan,NaN,8721897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,return on assets,property of $21 million and we deduct proceeds...,NaN,nan,NaN,8722106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,irr,"Course"" and ""Jeepers Creepers,"" MGM performed ...",NaN,nan,NaN,8722439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,roic,"Now, Slide 15 addresses our expec...",NaN,nan,NaN,8726899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80141,cost of debt,These costs are primarily comprised of interes...,NaN,nan,NaN,72711892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80303,interest rate,"To solve this issue, SNCR ultimately sold Intr...",NaN,nan,NaN,72743785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80304,irr,"With all that, we have a 44.5% gross irr since...",NaN,nan,NaN,72743785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80473,interest rate,We continue to see great demand for our produc...,NaN,nan,NaN,72800270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
entry_df = pd.read_excel("entryfiles_combined_v5_withparagraphs_andgvkey.xlsx")
entry_df

,Keywords,Paragraph,Date,gvkey,Title,Subtitle,Report
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,24040.0,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,65570.0,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,122096.0,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,6799.0,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965
4,irr,people per year. It has 300 retail stores inc...,NaN,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,8710501
...,...,...,...,...,...,...,...
80637,cost of equity,We're confident Stage 3 is well positioned for...,2021-09-07,29453.0,CHENIERE ENERGY INC,LNG.A - Event Transcript of Cheniere Energy In...,72918196
80638,cost of debt,Michael Webber - Webber Research & Advisory LL...,2021-09-07,29453.0,CHENIERE ENERGY INC,LNG.A - Event Brief of Cheniere Energy Inc con...,72918224
80639,irr,"On ESG, we just discussed. On performance, oka...",2021-09-07,296577.0,APERAM SA,APAM.AS - Event Transcript of Aperam SA confer...,72931747
80640,interest rate,Looking at Global Broking revenue by asset cla...,2021-09-07,241016.0,TP ICAP GROUP PLC,TCAPI.L - Event Transcript of TP ICAP Group PL...,72939535


In [33]:
def getTitles(row):
    if row["Title_x"] == "nan":#pd.isna(row["Title_x"]):
        return row["Title_y"]
    else:
        return row["Title_x"]

df = pd.read_excel("full_conf_calls_compustat_match.xlsx", engine = "openpyxl").drop("Unnamed: 0", axis = 1)
df["Title"] = df["Title"].astype(str)
full_df = df.copy()
full_df = full_df.merge(entry_df, on = ["Keywords", "Report"], how = "left")
full_df["Title"] = full_df.apply(getTitles, axis = 1)
full_df[full_df["Title"] == "nan"]

,Keywords,Paragraph_x,Date_x,Title_x,Subtitle_x,Report,gvkey_x,companyname,country,Compustat_Closest_Match,...,Hassan_Closest_Match_Similarity,Hassan_Closest_Match_Fullname,Hassan_Closest_Match_gvkey,Hassan_Closest_Match_Country,Paragraph_y,Date_y,gvkey_y,Title_y,Subtitle_y,Title


In [34]:
full_df

,Keywords,Paragraph_x,Date_x,Title_x,Subtitle_x,Report,gvkey_x,companyname,country,Compustat_Closest_Match,...,Hassan_Closest_Match_Similarity,Hassan_Closest_Match_Fullname,Hassan_Closest_Match_gvkey,Hassan_Closest_Match_Country,Paragraph_y,Date_y,gvkey_y,Title_y,Subtitle_y,Title
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,24040.0,"Ionis Pharmaceuticals, Inc.",United States,NaN,...,NaN,NaN,NaN,NaN,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,24040.0,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,65570.0,American Italian Pasta Company,United States,NaN,...,NaN,NaN,NaN,NaN,"35%, we anticipate margins in the fourth quar...",2002-07-24,65570.0,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,122096.0,"Insight Communications Company, Inc.",United States,NaN,...,NaN,NaN,NaN,NaN,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,122096.0,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,6799.0,KeySpan Corporation,United States,NaN,...,NaN,NaN,NaN,NaN,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,6799.0,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...
4,irr,people per year. It has 300 retail stores inc...,NaN,nan,NaN,8710501,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,people per year. It has 300 retail stores inc...,NaN,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,7970.0,NaN,United States,eversource energy,...,100.000000,Eversource Energy,7970.0,United States,NaN,NaN,NaN,NaN,NaN,EVERSOURCE ENERGY
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,7866.0,NaN,United States,new york times,...,100.000000,New York Times Co,7866.0,United States,NaN,NaN,NaN,NaN,NaN,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,285317.0,NaN,Portugal,ren redes energticas nacionais sgps,...,100.000000,REN Redes Energeticas Nacionais SGPS SA,285317.0,Portugal,NaN,NaN,NaN,NaN,NaN,REN REDES ENERGETICAS NACIONAIS SGPS SA
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,NaN,NaN,NaN,harel insurance financial services,...,62.295082,Selective Insurance Group Inc,17115.0,United States,NaN,NaN,NaN,NaN,NaN,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...


In [35]:
df, compustat_df = get_names(full_df, compustat_df, new_words)
df

,Keywords,Paragraph_x,Date_x,Title_x,Subtitle_x,Report,gvkey_x,companyname,country,Compustat_Closest_Match,...,gvkey_y,Title_y,Subtitle_y,Title,Name,Ticker,Other Information,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_Name_No_Punctuations_And_Removed_Words
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,24040.0,"Ionis Pharmaceuticals, Inc.",United States,NaN,...,24040.0,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,ISIS PHARMACEUTICALS (ISIS),ISIS,ISIS PHARMACEUTICALS DISCUSSES D,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,65570.0,American Italian Pasta Company,United States,NaN,...,65570.0,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,AMERICAN ITALIAN PASTA (PLB),PLB,Q3 2002 FINANCIAL RELEASE CONFE,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,122096.0,"Insight Communications Company, Inc.",United States,NaN,...,122096.0,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,INSIGHT COMMUNICATIONS (ICCI),ICCI,Q2 2002 FINANCIAL RELEASE CONF,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,6799.0,KeySpan Corporation,United States,NaN,...,6799.0,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,KEYSPAN CORP. (KSE),KSE,Q2 2002 FINANCIAL RELEASE CONFERENCE CAL,KEYSPAN CORP.,KEYSPAN CORP,KEYSPAN
4,irr,people per year. It has 300 retail stores inc...,NaN,nan,NaN,8710501,NaN,NaN,NaN,NaN,...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,SIMON PROPERTY GROUP INCORPORATED (SPG),SPG,Q2 2002 FINANCIAL RE,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY INCORPORATED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,7970.0,NaN,United States,eversource energy,...,NaN,NaN,NaN,EVERSOURCE ENERGY,EVERSOURCE ENERGY,NaN,NaN,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,7866.0,NaN,United States,new york times,...,NaN,NaN,NaN,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,NEW YORK TIMES COMPANY (THE),THE,PRELIMINARY TRANSCRIPT,NEW YORK TIMES COMPANY,NEW YORK TIMES COMPANY,NEW YORK TIMES
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,285317.0,NaN,Portugal,ren redes energticas nacionais sgps,...,NaN,NaN,NaN,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,NaN,NaN,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,NaN,NaN,NaN,harel insurance financial services,...,NaN,NaN,NaN,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,SELECTIVE INSURANCE GROUP (SIGI),SIGI,Q2 2003 FINANCIAL RELEASE C,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE


In [36]:
df[df["Title"] == "nan"]

,Keywords,Paragraph_x,Date_x,Title_x,Subtitle_x,Report,gvkey_x,companyname,country,Compustat_Closest_Match,...,gvkey_y,Title_y,Subtitle_y,Title,Name,Ticker,Other Information,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_Name_No_Punctuations_And_Removed_Words


In [39]:
def parseResults(results):
    c_firms, sims = [], []
    for c_firm, sim, _  in results:
        c_firms.append(c_firm)
        sims.append(sim)
    return c_firms, sims

In [41]:
full_df = df.copy()
small = full_df
small_dask = dd.from_pandas(small, npartitions = 100)
match_choices = compustat_df["companyname_No_Punctuations"]
results = small_dask["Cleaned_Name_No_Punctuations"].apply(lambda x: process.extractOne(x, match_choices, scorer = fuzz.ratio), meta = "str")
results = results.compute(scheduler = "processes")

In [42]:
results

0         (TELIOS PHARMACEUTICALS, 85.71428571428571, 30...
1         (AMERICAN ITALIAN PASTA COMPANY, 84.6153846153...
2         (GREENLIGHT COMMUNICATIONS, 85.1063829787234, ...
3                  (KEYERA CORP, 78.26086956521739, 306349)
4         (SIMON PROPERTY GROUP INC, 84.21052631578948, ...
                                ...                        
111455                    (EVERSOURCE ENERGY, 100.0, 29460)
111456    (THE NEW YORK TIMES COMPANY, 91.66666666666667...
111457    (REN  REDES ENERGÉTICAS NACIONAIS SGPS SA, 96....
111458    (SELECTIVE INSURANCE GROUP INC, 92.59259259259...
111459                           (TALANX AG, 100.0, 437627)
Length: 111460, dtype: object

In [43]:
c_firms, sims = [], []
our_firms = full_df["Cleaned_Name_No_Punctuations"]
for our_firm, j in zip(our_firms, results):
    c_firm, sim = j[0], j[1]
    c_firms.append(c_firm)
    sims.append(sim)
full_df["New_Cleaned_No_Punctuation_Compustat_Closest_Match"] = c_firms
full_df["New_Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity"] = sims
full_df

,Keywords,Paragraph_x,Date_x,Title_x,Subtitle_x,Report,gvkey_x,companyname,country,Compustat_Closest_Match,...,Subtitle_y,Title,Name,Ticker,Other Information,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_Name_No_Punctuations_And_Removed_Words,New_Cleaned_No_Punctuation_Compustat_Closest_Match,New_Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,24040.0,"Ionis Pharmaceuticals, Inc.",United States,NaN,...,Â,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,ISIS PHARMACEUTICALS (ISIS),ISIS,ISIS PHARMACEUTICALS DISCUSSES D,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,TELIOS PHARMACEUTICALS,85.714286
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,65570.0,American Italian Pasta Company,United States,NaN,...,Â,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,AMERICAN ITALIAN PASTA (PLB),PLB,Q3 2002 FINANCIAL RELEASE CONFE,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA COMPANY,84.615385
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,122096.0,"Insight Communications Company, Inc.",United States,NaN,...,Â,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,INSIGHT COMMUNICATIONS (ICCI),ICCI,Q2 2002 FINANCIAL RELEASE CONF,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,GREENLIGHT COMMUNICATIONS,85.106383
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,6799.0,KeySpan Corporation,United States,NaN,...,Â,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,KEYSPAN CORP. (KSE),KSE,Q2 2002 FINANCIAL RELEASE CONFERENCE CAL,KEYSPAN CORP.,KEYSPAN CORP,KEYSPAN,KEYERA CORP,78.260870
4,irr,people per year. It has 300 retail stores inc...,NaN,nan,NaN,8710501,NaN,NaN,NaN,NaN,...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,SIMON PROPERTY GROUP INCORPORATED (SPG),SPG,Q2 2002 FINANCIAL RE,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY INCORPORATED,SIMON PROPERTY GROUP INC,84.210526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,7970.0,NaN,United States,eversource energy,...,NaN,EVERSOURCE ENERGY,EVERSOURCE ENERGY,NaN,NaN,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,100.000000
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,7866.0,NaN,United States,new york times,...,NaN,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,NEW YORK TIMES COMPANY (THE),THE,PRELIMINARY TRANSCRIPT,NEW YORK TIMES COMPANY,NEW YORK TIMES COMPANY,NEW YORK TIMES,THE NEW YORK TIMES COMPANY,91.666667
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,285317.0,NaN,Portugal,ren redes energticas nacionais sgps,...,NaN,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,NaN,NaN,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS,REN REDES ENERGÉTICAS NACIONAIS SGPS SA,96.202532
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,NaN,NaN,NaN,harel insurance financial services,...,NaN,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,SELECTIVE INSURANCE GROUP (SIGI),SIGI,Q2 2003 FINANCIAL RELEASE C,SELECTIVE INSURANCE GROUP,SELECT

In [46]:
hassan_df = pd.read_csv("Hassanfile_raw_updated2019030_viewable.csv")
hassan_df = hassan_df[hassan_df["gvkey"].notnull()]
hassan_df = hassan_df[["gvkey", "company_name", "hqcountrycode", "ticker"]]
hassan_df = hassan_df.drop_duplicates("company_name")
hassan_df = hassan_df.reset_index(drop = True)
geo_df = pd.read_csv("ciqcountrygeo.csv")
non_null_geo_df = geo_df[geo_df["isocountry2"].notnull()]
country_dict = dict(non_null_geo_df.set_index("isocountry2")["country"])
hassan_df["country_name"] = hassan_df["hqcountrycode"].apply(lambda x: country_dict.get(x, np.nan))
hassan_df["company_name_No_Punctuations"] = hassan_df["company_name"].str.upper().str.split().apply(lambda x: remove_punc(x, new_words))
hassan_df

,gvkey,company_name,hqcountrycode,ticker,country_name,company_name_No_Punctuations
0,1004,AAR Corp,US,AIR,United States,AAR CORP
1,1013,CommScope Connectivity LLC,US,ADCT,United States,COMMSCOPE CONNECTIVITY LLC
2,1034,Alpharma LLC,US,ALO,United States,ALPHARMA LLC
3,1045,American Airlines Group Inc,US,AAL,United States,AMERICAN AIRLINES GROUP INC
4,1050,CECO Environmental Corp,US,CECE,United States,CECO ENVIRONMENTAL CORP
...,...,...,...,...,...,...
13353,347708,Darktrace PLC,GB,DARK,United Kingdom,DARKTRACE PLC
13354,348887,Bank Norwegian ASA,NO,BANO,Norway,BANK NORWEGIAN ASA
13355,349055,ABOUT YOU Holding SE,DE,YOU,Germany,ABOUT YOU HOLDING SE
13356,349404,PEXA Group Ltd,AU,PXA,Australia,PEXA GROUP LTD


In [47]:
match_choices = hassan_df["company_name_No_Punctuations"]
results = small_dask["Cleaned_Name_No_Punctuations"].apply(lambda x: process.extractOne(x, match_choices, scorer = fuzz.ratio), meta = "str")
results = results.compute(scheduler = "processes")
c_firms, sims = [], []
our_firms = full_df["Cleaned_Name_No_Punctuations"]
for our_firm, j in zip(our_firms, results):
    c_firm, sim = j[0], j[1]
    c_firms.append(c_firm)
    sims.append(sim)
full_df["New_Cleaned_No_Punctuation_Hassan_Closest_Match"] = c_firms
full_df["New_Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity"] = sims

In [48]:
full_df

,Keywords,Paragraph_x,Date_x,Title_x,Subtitle_x,Report,gvkey_x,companyname,country,Compustat_Closest_Match,...,Name,Ticker,Other Information,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_Name_No_Punctuations_And_Removed_Words,New_Cleaned_No_Punctuation_Compustat_Closest_Match,New_Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,New_Cleaned_No_Punctuation_Hassan_Closest_Match,New_Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,24040.0,"Ionis Pharmaceuticals, Inc.",United States,NaN,...,ISIS PHARMACEUTICALS (ISIS),ISIS,ISIS PHARMACEUTICALS DISCUSSES D,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,TELIOS PHARMACEUTICALS,85.714286,IONIS PHARMACEUTICALS INC,84.444444
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,65570.0,American Italian Pasta Company,United States,NaN,...,AMERICAN ITALIAN PASTA (PLB),PLB,Q3 2002 FINANCIAL RELEASE CONFE,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA COMPANY,84.615385,AMERICANAS SA,68.571429
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,122096.0,"Insight Communications Company, Inc.",United States,NaN,...,INSIGHT COMMUNICATIONS (ICCI),ICCI,Q2 2002 FINANCIAL RELEASE CONF,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,GREENLIGHT COMMUNICATIONS,85.106383,INVISIO COMMUNICATIONS AB,80.851064
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,6799.0,KeySpan Corporation,United States,NaN,...,KEYSPAN CORP. (KSE),KSE,Q2 2002 FINANCIAL RELEASE CONFERENCE CAL,KEYSPAN CORP.,KEYSPAN CORP,KEYSPAN,KEYERA CORP,78.260870,KEYSPAN CORP,100.000000
4,irr,people per year. It has 300 retail stores inc...,NaN,nan,NaN,8710501,NaN,NaN,NaN,NaN,...,SIMON PROPERTY GROUP INCORPORATED (SPG),SPG,Q2 2002 FINANCIAL RE,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY INCORPORATED,SIMON PROPERTY GROUP INC,84.210526,SIMON PROPERTY GROUP INC,84.210526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,7970.0,NaN,United States,eversource energy,...,EVERSOURCE ENERGY,NaN,NaN,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,100.000000,EVERSOURCE ENERGY,100.000000
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,7866.0,NaN,United States,new york times,...,NEW YORK TIMES COMPANY (THE),THE,PRELIMINARY TRANSCRIPT,NEW YORK TIMES COMPANY,NEW YORK TIMES COMPANY,NEW YORK TIMES,THE NEW YORK TIMES COMPANY,91.666667,NEW YORK TIMES CO,87.179487
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,285317.0,NaN,Portugal,ren redes energticas nacionais sgps,...,REN REDES ENERGETICAS NACIONAIS SGPS SA,NaN,NaN,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS,REN REDES ENERGÉTICAS NACIONAIS SGPS SA,96.202532,REN REDES ENERGETICAS NACIONAIS SGPS SA,100.000000
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,NaN,NaN,NaN,harel insurance financial services,...,SELECTIVE INSURANCE GROUP (SIGI),SIGI,Q2 2003 FINANCIAL RELEASE C,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE,SELECTIVE INSURANCE GROUP INC,92.592593,SEL

In [49]:
c_gvkeys, c_countries = [], []
for i in full_df["New_Cleaned_No_Punctuation_Compustat_Closest_Match"]:
    subset = compustat_df[compustat_df["companyname_No_Punctuations"] == i]
    gvkey = subset["gvkey"].iloc[0]
    country = subset["country"].iloc[0]
    c_gvkeys.append(gvkey)
    c_countries.append(country)
full_df["New_Cleaned_No_Punctuation_Compustat_Closest_Match_gvkey"] = c_gvkeys
full_df["New_Cleaned_No_Punctuation_Compustat_Closest_Match_country"] = c_countries

h_gvkeys, h_countries = [], []
for i in full_df["New_Cleaned_No_Punctuation_Hassan_Closest_Match"]:
    subset = hassan_df[hassan_df["company_name_No_Punctuations"] == i]
    #display(subset)
    #break
    gvkey = subset["gvkey"].iloc[0]
    country = subset["country_name"].iloc[0]
    h_gvkeys.append(gvkey)
    h_countries.append(country)
full_df["New_Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey"] = h_gvkeys
full_df["New_Cleaned_No_Punctuation_Hassan_Closest_Match_country"] = h_countries
full_df

,Keywords,Paragraph_x,Date_x,Title_x,Subtitle_x,Report,gvkey_x,companyname,country,Compustat_Closest_Match,...,Cleaned_Name_No_Punctuations,Cleaned_Name_No_Punctuations_And_Removed_Words,New_Cleaned_No_Punctuation_Compustat_Closest_Match,New_Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,New_Cleaned_No_Punctuation_Hassan_Closest_Match,New_Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity,New_Cleaned_No_Punctuation_Compustat_Closest_Match_gvkey,New_Cleaned_No_Punctuation_Compustat_Closest_Match_country,New_Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey,New_Cleaned_No_Punctuation_Hassan_Closest_Match_country
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,24040.0,"Ionis Pharmaceuticals, Inc.",United States,NaN,...,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,TELIOS PHARMACEUTICALS,85.714286,IONIS PHARMACEUTICALS INC,84.444444,25067.0,United States,24040,United States
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,65570.0,American Italian Pasta Company,United States,NaN,...,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA COMPANY,84.615385,AMERICANAS SA,68.571429,65570.0,United States,163677,Brazil
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,122096.0,"Insight Communications Company, Inc.",United States,NaN,...,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,GREENLIGHT COMMUNICATIONS,85.106383,INVISIO COMMUNICATIONS AB,80.851064,63727.0,Canada,278126,Sweden
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,6799.0,KeySpan Corporation,United States,NaN,...,KEYSPAN CORP,KEYSPAN,KEYERA CORP,78.260870,KEYSPAN CORP,100.000000,154135.0,Canada,6799,United States
4,irr,people per year. It has 300 retail stores inc...,NaN,nan,NaN,8710501,NaN,NaN,NaN,NaN,...,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY INCORPORATED,SIMON PROPERTY GROUP INC,84.210526,SIMON PROPERTY GROUP INC,84.210526,29389.0,United States,29389,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,7970.0,NaN,United States,eversource energy,...,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,100.000000,EVERSOURCE ENERGY,100.000000,7970.0,United States,7970,United States
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,7866.0,NaN,United States,new york times,...,NEW YORK TIMES COMPANY,NEW YORK TIMES,THE NEW YORK TIMES COMPANY,91.666667,NEW YORK TIMES CO,87.179487,7866.0,United States,7866,United States
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,285317.0,NaN,Portugal,ren redes energticas nacionais sgps,...,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS,REN REDES ENERGÉTICAS NACIONAIS SGPS SA,96.202532,REN REDES ENERGETICAS NACIONAIS SGPS SA,100.000000,285317.0,Portugal,285317,Portugal
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,NaN,NaN,NaN,harel insurance financial services,...,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE,SELECTIVE INSURANCE GROUP INC,92.592593,SELECTIVE INSURANCE GROUP INC,92.592593,17115.0,United States,17115,United States


In [50]:
def choose_gvkey(row, threshold = 95):
    s1 = "New_Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity"
    s2 = "New_Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity"
    
    if (row[s1] >= threshold) or (row[s2] >= threshold):
        if row[s1] >= row[s2]:
            gvkey = row["New_Cleaned_No_Punctuation_Compustat_Closest_Match_gvkey"]
            country = row["New_Cleaned_No_Punctuation_Compustat_Closest_Match_country"]
            return (gvkey, country)
        else:
            gvkey = row["New_Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey"]
            country = row["New_Cleaned_No_Punctuation_Hassan_Closest_Match_country"]
            return (gvkey, country)
    
    else:
        return (np.nan, np.nan)

t = full_df.apply(choose_gvkey, axis = 1)
new_gvkeys = [i[0] for i in t]
new_countries = [i[1] for i in t]
full_df["final_gvkey"] = new_gvkeys
full_df["final_country"] = new_countries
full_df

,Keywords,Paragraph_x,Date_x,Title_x,Subtitle_x,Report,gvkey_x,companyname,country,Compustat_Closest_Match,...,New_Cleaned_No_Punctuation_Compustat_Closest_Match,New_Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,New_Cleaned_No_Punctuation_Hassan_Closest_Match,New_Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity,New_Cleaned_No_Punctuation_Compustat_Closest_Match_gvkey,New_Cleaned_No_Punctuation_Compustat_Closest_Match_country,New_Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey,New_Cleaned_No_Punctuation_Hassan_Closest_Match_country,final_gvkey,final_country
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,24040.0,"Ionis Pharmaceuticals, Inc.",United States,NaN,...,TELIOS PHARMACEUTICALS,85.714286,IONIS PHARMACEUTICALS INC,84.444444,25067.0,United States,24040,United States,NaN,NaN
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,65570.0,American Italian Pasta Company,United States,NaN,...,AMERICAN ITALIAN PASTA COMPANY,84.615385,AMERICANAS SA,68.571429,65570.0,United States,163677,Brazil,NaN,NaN
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,122096.0,"Insight Communications Company, Inc.",United States,NaN,...,GREENLIGHT COMMUNICATIONS,85.106383,INVISIO COMMUNICATIONS AB,80.851064,63727.0,Canada,278126,Sweden,NaN,NaN
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,6799.0,KeySpan Corporation,United States,NaN,...,KEYERA CORP,78.260870,KEYSPAN CORP,100.000000,154135.0,Canada,6799,United States,6799.0,United States
4,irr,people per year. It has 300 retail stores inc...,NaN,nan,NaN,8710501,NaN,NaN,NaN,NaN,...,SIMON PROPERTY GROUP INC,84.210526,SIMON PROPERTY GROUP INC,84.210526,29389.0,United States,29389,United States,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,7970.0,NaN,United States,eversource energy,...,EVERSOURCE ENERGY,100.000000,EVERSOURCE ENERGY,100.000000,7970.0,United States,7970,United States,7970.0,United States
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,7866.0,NaN,United States,new york times,...,THE NEW YORK TIMES COMPANY,91.666667,NEW YORK TIMES CO,87.179487,7866.0,United States,7866,United States,NaN,NaN
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,285317.0,NaN,Portugal,ren redes energticas nacionais sgps,...,REN REDES ENERGÉTICAS NACIONAIS SGPS SA,96.202532,REN REDES ENERGETICAS NACIONAIS SGPS SA,100.000000,285317.0,Portugal,285317,Portugal,285317.0,Portugal
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,NaN,NaN,NaN,harel insurance financial services,...,SELECTIVE INSURANCE GROUP INC,92.592593,SELECTIVE INSURANCE GROUP INC,92.592593,17115.0,United States,17115,United States,NaN,NaN


In [51]:
full_df = full_df.rename({"Paragraph_x": "Paragraph", "Date_x": "Date", "Subtitle_x": "Subtitle"}, axis = 1)
full_df.columns

Index(['Keywords', 'Paragraph', 'Date', 'Title_x', 'Subtitle', 'Report',
       'gvkey_x', 'companyname', 'country', 'Compustat_Closest_Match',
       'Compustat_Closest_Match_Similarity',
       'Compustat_Closest_Match_Fullname', 'Compustat_Closest_Match_gvkey',
       'Compustat_Closest_Match_Country', 'Hassan_Closest_Match',
       'Hassan_Closest_Match_Similarity', 'Hassan_Closest_Match_Fullname',
       'Hassan_Closest_Match_gvkey', 'Hassan_Closest_Match_Country',
       'Paragraph_y', 'Date_y', 'gvkey_y', 'Title_y', 'Subtitle_y', 'Title',
       'Name', 'Ticker', 'Other Information', 'Cleaned_Name',
       'Cleaned_Name_No_Punctuations',
       'Cleaned_Name_No_Punctuations_And_Removed_Words',
       'New_Cleaned_No_Punctuation_Compustat_Closest_Match',
       'New_Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity',
       'New_Cleaned_No_Punctuation_Hassan_Closest_Match',
       'New_Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity',
       'New_Cleaned_No_Punctua

In [52]:
c_firms_fullnames, h_firms_fullnames = [], []
for c_firm, h_firm in zip(full_df["New_Cleaned_No_Punctuation_Compustat_Closest_Match"], full_df["New_Cleaned_No_Punctuation_Hassan_Closest_Match"]):
    c_subset = compustat_df[compustat_df["companyname_No_Punctuations"] == c_firm]
    h_subset = hassan_df[hassan_df["company_name_No_Punctuations"] == h_firm]
    c_fullname = c_subset["companyname"].iloc[0]
    h_fullname = h_subset["company_name"].iloc[0]
    c_firms_fullnames.append(c_fullname)
    h_firms_fullnames.append(h_fullname)
    
full_df["New_Cleaned_No_Punctuation_Compustat_Closest_Match_Fullname"] = c_firms_fullnames
full_df["New_Cleaned_No_Punctuation_Hassan_Closest_Match_Fullname"] = h_firms_fullnames
full_df

,Keywords,Paragraph,Date,Title_x,Subtitle,Report,gvkey_x,companyname,country,Compustat_Closest_Match,...,New_Cleaned_No_Punctuation_Hassan_Closest_Match,New_Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity,New_Cleaned_No_Punctuation_Compustat_Closest_Match_gvkey,New_Cleaned_No_Punctuation_Compustat_Closest_Match_country,New_Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey,New_Cleaned_No_Punctuation_Hassan_Closest_Match_country,final_gvkey,final_country,New_Cleaned_No_Punctuation_Compustat_Closest_Match_Fullname,New_Cleaned_No_Punctuation_Hassan_Closest_Match_Fullname
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,24040.0,"Ionis Pharmaceuticals, Inc.",United States,NaN,...,IONIS PHARMACEUTICALS INC,84.444444,25067.0,United States,24040,United States,NaN,NaN,Telios Pharmaceuticals,Ionis Pharmaceuticals Inc
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,65570.0,American Italian Pasta Company,United States,NaN,...,AMERICANAS SA,68.571429,65570.0,United States,163677,Brazil,NaN,NaN,American Italian Pasta Company,Americanas SA
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,122096.0,"Insight Communications Company, Inc.",United States,NaN,...,INVISIO COMMUNICATIONS AB,80.851064,63727.0,Canada,278126,Sweden,NaN,NaN,Greenlight Communications,Invisio Communications AB
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,6799.0,KeySpan Corporation,United States,NaN,...,KEYSPAN CORP,100.000000,154135.0,Canada,6799,United States,6799.0,United States,Keyera Corp.,Keyspan Corp
4,irr,people per year. It has 300 retail stores inc...,NaN,nan,NaN,8710501,NaN,NaN,NaN,NaN,...,SIMON PROPERTY GROUP INC,84.210526,29389.0,United States,29389,United States,NaN,NaN,"Simon Property Group, Inc.",Simon Property Group Inc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,7970.0,NaN,United States,eversource energy,...,EVERSOURCE ENERGY,100.000000,7970.0,United States,7970,United States,7970.0,United States,Eversource Energy,Eversource Energy
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,7866.0,NaN,United States,new york times,...,NEW YORK TIMES CO,87.179487,7866.0,United States,7866,United States,NaN,NaN,The New York Times Company,New York Times Co
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,285317.0,NaN,Portugal,ren redes energticas nacionais sgps,...,REN REDES ENERGETICAS NACIONAIS SGPS SA,100.000000,285317.0,Portugal,285317,Portugal,285317.0,Portugal,"REN - Redes Energéticas Nacionais, SGPS, S.A.",REN Redes Energeticas Nacionais SGPS SA
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,NaN,NaN,NaN,harel insurance financial services,...,SELECTIVE INSURANCE GROUP INC,92.592593,17115.0,United States,17115,United States,NaN,NaN,"Selective Insurance Group, Inc.",Selective Insurance Group Inc


In [54]:
cols = ["Keywords", "Paragraph", "Date", "Title", "Subtitle", "Report",
       'Cleaned_Name', 'Cleaned_Name_No_Punctuations', 'New_Cleaned_No_Punctuation_Compustat_Closest_Match',
       'New_Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity',
       'New_Cleaned_No_Punctuation_Compustat_Closest_Match_gvkey',
        "New_Cleaned_No_Punctuation_Compustat_Closest_Match_Fullname",
       'New_Cleaned_No_Punctuation_Compustat_Closest_Match_country', 
       'New_Cleaned_No_Punctuation_Hassan_Closest_Match',
       'New_Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity',
       'New_Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey',
        "New_Cleaned_No_Punctuation_Hassan_Closest_Match_Fullname",
       'New_Cleaned_No_Punctuation_Hassan_Closest_Match_country', 'final_gvkey', 'final_country']

new_full_df = full_df[cols]
new_names = []
for colname in new_full_df.columns:
    if colname[:4] == "New_":
        new_names.append(colname[4:])
    else:
        new_names.append(colname)
new_full_df.columns = new_names
new_full_df

,Keywords,Paragraph,Date,Title,Subtitle,Report,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_No_Punctuation_Compustat_Closest_Match,Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,Cleaned_No_Punctuation_Compustat_Closest_Match_gvkey,Cleaned_No_Punctuation_Compustat_Closest_Match_Fullname,Cleaned_No_Punctuation_Compustat_Closest_Match_country,Cleaned_No_Punctuation_Hassan_Closest_Match,Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity,Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey,Cleaned_No_Punctuation_Hassan_Closest_Match_Fullname,Cleaned_No_Punctuation_Hassan_Closest_Match_country,final_gvkey,final_country
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,TELIOS PHARMACEUTICALS,85.714286,25067.0,Telios Pharmaceuticals,United States,IONIS PHARMACEUTICALS INC,84.444444,24040,Ionis Pharmaceuticals Inc,United States,NaN,NaN
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA COMPANY,84.615385,65570.0,American Italian Pasta Company,United States,AMERICANAS SA,68.571429,163677,Americanas SA,Brazil,NaN,NaN
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,GREENLIGHT COMMUNICATIONS,85.106383,63727.0,Greenlight Communications,Canada,INVISIO COMMUNICATIONS AB,80.851064,278126,Invisio Communications AB,Sweden,NaN,NaN
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,KEYSPAN CORP.,KEYSPAN CORP,KEYERA CORP,78.260870,154135.0,Keyera Corp.,Canada,KEYSPAN CORP,100.000000,6799,Keyspan Corp,United States,6799.0,United States
4,irr,people per year. It has 300 retail stores inc...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,8710501,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INC,84.210526,29389.0,"Simon Property Group, Inc.",United States,SIMON PROPERTY GROUP INC,84.210526,29389,Simon Property Group Inc,United States,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,100.000000,7970.0,Eversource Energy,United States,EVERSOURCE ENERGY,100.000000,7970,Eversource Energy,United States,7970.0,United States
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,NEW YORK TIMES COMPANY,NEW YORK TIMES COMPANY,THE NEW YORK TIMES COMPANY,91.666667,7866.0,The New York Times Company,United States,NEW YORK TIMES CO,87.179487,7866,New York Times Co,United States,NaN,NaN
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGÉTICAS NACIONAIS SGPS SA,96.202532,285317.0,"REN - Redes Energéticas Nacionais, SGPS, S.A.",Portugal,REN REDES ENERGETICAS NACIONAIS SGPS SA,100.000000,285317,REN Redes Energeticas Nacionais SGPS SA,Portugal,285317.0,Portugal
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP INC,92.592593,17115.0,"Selective Insurance Group, Inc.",United States,SELECTIVE INSURANCE GROUP INC,92.592593,17115,Selective Insurance Group Inc,United States,NaN,NaN


In [56]:
#new_full_df.to_excel("updated_full_conf_calls_match.xlsx", index = False)

In [55]:
new_matched = new_full_df[new_full_df["final_gvkey"].notnull()]
new_matched#.to_excel("updated_matched_conf_calls_match.xlsx", index = False)

,Keywords,Paragraph,Date,Title,Subtitle,Report,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_No_Punctuation_Compustat_Closest_Match,Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,Cleaned_No_Punctuation_Compustat_Closest_Match_gvkey,Cleaned_No_Punctuation_Compustat_Closest_Match_Fullname,Cleaned_No_Punctuation_Compustat_Closest_Match_country,Cleaned_No_Punctuation_Hassan_Closest_Match,Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity,Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey,Cleaned_No_Punctuation_Hassan_Closest_Match_Fullname,Cleaned_No_Punctuation_Hassan_Closest_Match_country,final_gvkey,final_country
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,KEYSPAN CORP.,KEYSPAN CORP,KEYERA CORP,78.260870,154135.0,Keyera Corp.,Canada,KEYSPAN CORP,100.000000,6799,Keyspan Corp,United States,6799.0,United States
5,interest rate,receivables over 30 day delinquent also decre...,2002-07-30,ADVANTA CORP. (ADVNB) - Q2 2002 FINANCIAL RELE...,Â,8710548,ADVANTA CORP.,ADVANTA CORP,ADVANTA CORP,100.000000,12108.0,Advanta Corp.,United States,ADVANTA CORP,100.000000,12108,Advanta Corp,United States,12108.0,United States
6,interest rate,MR. GARY MCDANIEL: While we feel good about th...,2002-08-01,"CHATEAU COMMUNITIES, INC. (CPJ) - Q2 2002 FINA...",Â,8710632,"CHATEAU COMMUNITIES, INC.",CHATEAU COMMUNITIES INC,CHATEAU COMMUNITIES INC,100.000000,29233.0,"Chateau Communities, Inc.",United States,CENTURY COMMUNITIES INC,82.608696,18004,Century Communities Inc,United States,29233.0,United States
8,discount rate,As I mentioned on the last quarter conference ...,2003-01-22,"CYTEC INDUSTRIES, INC. (CYT) - Q4 2002 FINANCI...",Â,8720941,"CYTEC INDUSTRIES, INC.",CYTEC INDUSTRIES INC,CYTEC INDUSTRIES INC,100.000000,29511.0,Cytec Industries Inc.,United States,CYTEC INDUSTRIES INC,100.000000,29511,Cytec Industries Inc,United States,29511.0,United States
9,discount rate,Let me update you on Cytec's pension accountin...,2003-01-22,"CYTEC INDUSTRIES, INC. (CYT) - Q4 2002 FINANCI...",Â,8720951,"CYTEC INDUSTRIES, INC.",CYTEC INDUSTRIES INC,CYTEC INDUSTRIES INC,100.000000,29511.0,Cytec Industries Inc.,United States,CYTEC INDUSTRIES INC,100.000000,29511,Cytec Industries Inc,United States,29511.0,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111453,CAPM,Evan Calio - Morgan Stanley - Analyst Understo...,2013-04-26,CHEVRON CORP,CVX.N - Event Brief of Chevron Corp conference...,21812131,CHEVRON CORP,CHEVRON CORP,CHIRON CORP,86.956522,3011.0,"Chiron, Corp.",United States,CHEVRON CORP,100.000000,2991,Chevron Corp,United States,2991.0,United States
111454,CAPM,Turning from the states to the FERC informatio...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Transcript of Eversource Energy c...,68930082,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,100.000000,7970.0,Eversource Energy,United States,EVERSOURCE ENERGY,100.000000,7970,Eversource Energy,United States,7970.0,United States
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,100.000000,7970.0,Eversource Energy,United States,EVERSOURCE ENERGY,100.000000,7970,Eversource Energy,United States,7970.0,United States
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGÉTICAS NACIONAIS SGPS SA,96.202532,285317.0,"REN - Redes Energéticas Nacionais, SGPS, S.A.",Portugal,REN REDES ENERGETICAS NACIONAIS SGPS SA,100.000000,285317,REN Redes Energeticas Nacionais SGPS SA,Portugal,285317.0,Portugal


In [57]:
new_unmatched = new_full_df[new_full_df["final_gvkey"].isnull()]
new_unmatched#.to_excel("updated_unmatched_conf_calls_match.xlsx", index = False)

,Keywords,Paragraph,Date,Title,Subtitle,Report,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_No_Punctuation_Compustat_Closest_Match,Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,Cleaned_No_Punctuation_Compustat_Closest_Match_gvkey,Cleaned_No_Punctuation_Compustat_Closest_Match_Fullname,Cleaned_No_Punctuation_Compustat_Closest_Match_country,Cleaned_No_Punctuation_Hassan_Closest_Match,Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity,Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey,Cleaned_No_Punctuation_Hassan_Closest_Match_Fullname,Cleaned_No_Punctuation_Hassan_Closest_Match_country,final_gvkey,final_country
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,TELIOS PHARMACEUTICALS,85.714286,25067.0,Telios Pharmaceuticals,United States,IONIS PHARMACEUTICALS INC,84.444444,24040,Ionis Pharmaceuticals Inc,United States,NaN,NaN
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA COMPANY,84.615385,65570.0,American Italian Pasta Company,United States,AMERICANAS SA,68.571429,163677,Americanas SA,Brazil,NaN,NaN
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,GREENLIGHT COMMUNICATIONS,85.106383,63727.0,Greenlight Communications,Canada,INVISIO COMMUNICATIONS AB,80.851064,278126,Invisio Communications AB,Sweden,NaN,NaN
4,irr,people per year. It has 300 retail stores inc...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,8710501,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INC,84.210526,29389.0,"Simon Property Group, Inc.",United States,SIMON PROPERTY GROUP INC,84.210526,29389,Simon Property Group Inc,United States,NaN,NaN
7,interest rate,in the second−quarter '02 compared to 7 cents...,2002-08-14,"HPSC, INC. (HDR) - Q2 2002 FINANCIAL RELEASE C...",Â,8712139,"HPSC, INC.",HPSC INC,HPVC INC,87.500000,166428.0,HPVC Inc.,Canada,HP INC,85.714286,5606,HP Inc,United States,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111437,OCC,A. Highlights: 1. Executing synergy a...,2018-01-29,WRK - EVENT BRIEF OF WRKCO INC CONFERENCE CALL...,WRK - Event Brief of WRKCO INC conference call...,64220431,WRK,WRK,W KRUK SA,50.000000,254252.0,W. Kruk S.A.,Poland,WRKCO INC,50.000000,29830,WRKCO Inc,United States,NaN,NaN
111438,OCC,Transportation costs are being negatively impa...,2018-01-29,WRK - EVENT TRANSCRIPT OF WRKCO INC CONFERENCE...,WRK - Event Transcript of WRKCO INC conference...,64220414,WRK,WRK,W KRUK SA,50.000000,254252.0,W. Kruk S.A.,Poland,WRKCO INC,50.000000,29830,WRKCO Inc,United States,NaN,NaN
111439,OCC,Christopher David Manuel - Wells Fargo Securit...,2018-01-29,WRK - EVENT TRANSCRIPT OF WRKCO INC CONFERENCE...,WRK - Event Transcript of WRKCO INC conference...,64220414,WRK,WRK,W KRUK SA,50.000000,254252.0,W. Kruk S.A.,Poland,WRKCO INC,50.000000,29830,WRKCO Inc,United States,NaN,NaN
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,NEW YORK TIMES COMPANY,NEW YORK TIMES COMPANY,THE NEW YORK TIMES COMPANY,91.666667,7866.0,The New York Times Company,United States,NEW YORK TIMES CO,87.179487,7866,New York Times Co,United States,NaN,NaN


### Updating after manual work:

In [83]:
filled_c = pd.read_excel("Filled_Updated_CC_Compustat_FuzzyMatchCandidates.xlsx")
filled_c = filled_c[filled_c["Correct"] == 1]
filled_c

,CC Data Index,CC Firm,Compustat Firm,Similarity,Correct,Notes
1,0,ISIS PHARMACEUTICALS,IONIS PHARMACEUTICALS INC,84.444444,1,NaN
5,1,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA COMPANY,84.615385,1,NaN
11,4,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INC,84.210526,1,NaN
19,13,COOPER INDUSTRIES INC,COOPER INDUSTRIES PLC,90.476190,1,NaN
24,16,ELI LILLY CO,ELI LILLY AND COMPANY,76.470588,1,NaN
...,...,...,...,...,...,...
9510,110040,ABITIBICONSOLIDATED,ABITIBI CONSOLIDATED INC,88.372093,1,NaN
9512,110135,CARAUSTAR INDUSTRIES,CARAUSTAR INDUSTRIES INC,90.909091,1,NaN
9523,110202,CHARTER ONE FINANCIAL,CHARTER ONE FINANCIAL INC,91.304348,1,NaN
9527,110365,GRAPHIC PACKAGING INTL,GRAPHIC PACKAGING INTERNATIONAL LLC,77.192982,1,NaN


In [84]:
filled_h = pd.read_excel("Filled_Updated_CC_Hassan_FuzzyMatchCandidates.xlsx")
filled_h = filled_h[filled_h["Correct"] == 1]
filled_h

,CC Data Index,CC Firm,Hassan Firm,Similarity,Correct,Notes
1,2,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS COMPANY LLC,78.571429,1.0,NaN
12,43,GANNETT COMPANY,GANNETT CO INC,75.862069,1.0,NaN
28,79,COCA COLA ENTERPRISES,COCACOLA ENTERPRISES INC,88.888889,1.0,NaN
41,136,HUNTINGTON BANCSHARES,HUNTINGTON BANCSHARES INC,91.304348,1.0,NaN
43,152,YELLOW CORPORATION,YELLOW CORP,75.862069,1.0,NaN
...,...,...,...,...,...,...
3749,106579,EVENT TRANSCRIPT OF FORMER CHARTER COMMUNICAT...,FORMER CHARTER COMMUNICATIONS PARENT INC,80.000000,1.0,NaN
3762,107279,CORPORATE OFFICE PROPERTIES,CORPORATE OFFICE PROPERTIES TRUST,90.000000,1.0,NaN
3774,110199,CCL INDUSTRIES INCORPORATED,CCL INDUSTRIES INC,80.000000,1.0,NaN
3780,110810,PMI GROUP,PMI GROUP INC,81.818182,1.0,NaN


In [85]:
comp_dict = dict(filled_c.set_index("CC Firm")["Compustat Firm"])
hassan_dict = dict(filled_h.set_index("CC Firm")["Hassan Firm"])

In [86]:
#drop_columns = ["new_gvkeys", "new_country", "final_gvkey", "final_country"]
#df = pd.read_excel("updated_full_conf_calls_match.xlsx")
df = new_full_df.copy()
types, closest_firms = [], []
for index, row in df.iterrows():
    gvkey = row["final_gvkey"]
    if not pd.isna(gvkey):
        types.append(np.nan)
        closest_firms.append(np.nan)
        continue
    
    cc_firm = row["Cleaned_Name_No_Punctuations"]
    comp_firm = comp_dict.get(cc_firm, -99999)
    if comp_firm == -99999:
        hassan_firm = hassan_dict.get(cc_firm, -99999)
        try:
            hassan_firm = hassan_firm.iloc[0]
        except:
            pass
        if hassan_firm == -99999:
            types.append(np.nan)
            closest_firms.append(np.nan)
        else:
            types.append("Hassan")
            closest_firms.append(hassan_firm)
    else:
        types.append("Compustat")
        closest_firms.append(comp_firm)

In [87]:
df["Manual_Match_Firm"] = closest_firms
df["Manual_Match_Type"] = types
df

,Keywords,Paragraph,Date,Title,Subtitle,Report,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_No_Punctuation_Compustat_Closest_Match,Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,...,Cleaned_No_Punctuation_Compustat_Closest_Match_country,Cleaned_No_Punctuation_Hassan_Closest_Match,Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity,Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey,Cleaned_No_Punctuation_Hassan_Closest_Match_Fullname,Cleaned_No_Punctuation_Hassan_Closest_Match_country,final_gvkey,final_country,Manual_Match_Firm,Manual_Match_Type
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,TELIOS PHARMACEUTICALS,85.714286,...,United States,IONIS PHARMACEUTICALS INC,84.444444,24040,Ionis Pharmaceuticals Inc,United States,NaN,NaN,IONIS PHARMACEUTICALS INC,Compustat
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA COMPANY,84.615385,...,United States,AMERICANAS SA,68.571429,163677,Americanas SA,Brazil,NaN,NaN,AMERICAN ITALIAN PASTA COMPANY,Compustat
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,GREENLIGHT COMMUNICATIONS,85.106383,...,Canada,INVISIO COMMUNICATIONS AB,80.851064,278126,Invisio Communications AB,Sweden,NaN,NaN,INSIGHT COMMUNICATIONS COMPANY LLC,Hassan
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,KEYSPAN CORP.,KEYSPAN CORP,KEYERA CORP,78.260870,...,Canada,KEYSPAN CORP,100.000000,6799,Keyspan Corp,United States,6799.0,United States,NaN,NaN
4,irr,people per year. It has 300 retail stores inc...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,8710501,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INC,84.210526,...,United States,SIMON PROPERTY GROUP INC,84.210526,29389,Simon Property Group Inc,United States,NaN,NaN,SIMON PROPERTY GROUP INC,Compustat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,100.000000,...,United States,EVERSOURCE ENERGY,100.000000,7970,Eversource Energy,United States,7970.0,United States,NaN,NaN
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,NEW YORK TIMES COMPANY,NEW YORK TIMES COMPANY,THE NEW YORK TIMES COMPANY,91.666667,...,United States,NEW YORK TIMES CO,87.179487,7866,New York Times Co,United States,NaN,NaN,THE NEW YORK TIMES COMPANY,Compustat
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGÉTICAS NACIONAIS SGPS SA,96.202532,...,Portugal,REN REDES ENERGETICAS NACIONAIS SGPS SA,100.000000,285317,REN Redes Energeticas Nacionais SGPS SA,Portugal,285317.0,Portugal,NaN,NaN
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP INC,92.592593,...,United States,SELECTIVE INSURANCE GROUP INC,92.592593,17115,Selective Insurance Group Inc,United States,NaN,NaN,SELECTIVE INSURANCE GROUP INC,Compustat


In [88]:
gvkeys, countries, fullnames = [], [], []
for index, row in df.iterrows():
    final_gvkey = row["final_gvkey"]
    if not pd.isna(final_gvkey):
        gvkeys.append(final_gvkey)
        countries.append(row["final_country"])
        continue

    closest_firm = row["Manual_Match_Firm"]
    if pd.isna(closest_firm):
        gvkey, country, fullname = np.nan, np.nan, np.nan
    else:
        dataset = row["Manual_Match_Type"]
        if dataset == "Compustat":
            subset = compustat_df[compustat_df["companyname_No_Punctuations"] == closest_firm]
            gvkey, country, fullname = subset["gvkey"].iloc[0], subset["country"].iloc[0], subset["companyname"].iloc[0]
        elif dataset == "Hassan":
            subset = hassan_df[hassan_df["company_name_No_Punctuations"] == closest_firm]
            gvkey, country, fullname = subset["gvkey"].iloc[0], subset["country_name"].iloc[0], subset["company_name"].iloc[0]
    gvkeys.append(gvkey)
    countries.append(country)
    fullnames.append(fullname)

df["new_gvkeys"], df["new_country"] = gvkeys, country
df

,Keywords,Paragraph,Date,Title,Subtitle,Report,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_No_Punctuation_Compustat_Closest_Match,Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,...,Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity,Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey,Cleaned_No_Punctuation_Hassan_Closest_Match_Fullname,Cleaned_No_Punctuation_Hassan_Closest_Match_country,final_gvkey,final_country,Manual_Match_Firm,Manual_Match_Type,new_gvkeys,new_country
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,TELIOS PHARMACEUTICALS,85.714286,...,84.444444,24040,Ionis Pharmaceuticals Inc,United States,NaN,NaN,IONIS PHARMACEUTICALS INC,Compustat,24040.0,United States
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA COMPANY,84.615385,...,68.571429,163677,Americanas SA,Brazil,NaN,NaN,AMERICAN ITALIAN PASTA COMPANY,Compustat,65570.0,United States
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,GREENLIGHT COMMUNICATIONS,85.106383,...,80.851064,278126,Invisio Communications AB,Sweden,NaN,NaN,INSIGHT COMMUNICATIONS COMPANY LLC,Hassan,122096.0,United States
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,KEYSPAN CORP.,KEYSPAN CORP,KEYERA CORP,78.260870,...,100.000000,6799,Keyspan Corp,United States,6799.0,United States,NaN,NaN,6799.0,United States
4,irr,people per year. It has 300 retail stores inc...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,8710501,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INC,84.210526,...,84.210526,29389,Simon Property Group Inc,United States,NaN,NaN,SIMON PROPERTY GROUP INC,Compustat,29389.0,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,100.000000,...,100.000000,7970,Eversource Energy,United States,7970.0,United States,NaN,NaN,7970.0,United States
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,NEW YORK TIMES COMPANY,NEW YORK TIMES COMPANY,THE NEW YORK TIMES COMPANY,91.666667,...,87.179487,7866,New York Times Co,United States,NaN,NaN,THE NEW YORK TIMES COMPANY,Compustat,7866.0,United States
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGÉTICAS NACIONAIS SGPS SA,96.202532,...,100.000000,285317,REN Redes Energeticas Nacionais SGPS SA,Portugal,285317.0,Portugal,NaN,NaN,285317.0,United States
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP INC,92.592593,...,92.592593,17115,Selective Insurance Group Inc,United States,NaN,NaN,SELECTIVE INSURANCE GROUP INC,Compustat,17115.0,United States


In [89]:
gvkeys, countries, fullnames = [], [], []
for i in df["new_gvkeys"]:
    if pd.isna(i):
        gvkeys.append(np.nan)
        countries.append(np.nan)
        fullnames.append(np.nan)
        continue
    subset = compustat_df[compustat_df["gvkey"] == i]
    if len(subset) < 1:
        # Only Hassan Territory
        gvkeys.append(np.nan)
        countries.append(np.nan)
        fullnames.append(np.nan)
        continue
    gvkey, country, fullname = subset["gvkey"].iloc[0], subset["country"].iloc[0], subset["companyname"].iloc[0]
    gvkeys.append(i)
    countries.append(country)
    fullnames.append(fullname)
df["manual_gvkey"], df["manual_country"], df["manual_fullnames"] = gvkeys, countries, fullnames
df

,Keywords,Paragraph,Date,Title,Subtitle,Report,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_No_Punctuation_Compustat_Closest_Match,Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,...,Cleaned_No_Punctuation_Hassan_Closest_Match_country,final_gvkey,final_country,Manual_Match_Firm,Manual_Match_Type,new_gvkeys,new_country,manual_gvkey,manual_country,manual_fullnames
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,TELIOS PHARMACEUTICALS,85.714286,...,United States,NaN,NaN,IONIS PHARMACEUTICALS INC,Compustat,24040.0,United States,24040.0,United States,"Ionis Pharmaceuticals, Inc."
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA COMPANY,84.615385,...,Brazil,NaN,NaN,AMERICAN ITALIAN PASTA COMPANY,Compustat,65570.0,United States,65570.0,United States,American Italian Pasta Company
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,GREENLIGHT COMMUNICATIONS,85.106383,...,Sweden,NaN,NaN,INSIGHT COMMUNICATIONS COMPANY LLC,Hassan,122096.0,United States,122096.0,United States,"Insight Communications Company, Inc."
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,KEYSPAN CORP.,KEYSPAN CORP,KEYERA CORP,78.260870,...,United States,6799.0,United States,NaN,NaN,6799.0,United States,6799.0,United States,KeySpan Corporation
4,irr,people per year. It has 300 retail stores inc...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,8710501,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INC,84.210526,...,United States,NaN,NaN,SIMON PROPERTY GROUP INC,Compustat,29389.0,United States,29389.0,United States,"Simon Property Group, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,100.000000,...,United States,7970.0,United States,NaN,NaN,7970.0,United States,7970.0,United States,Eversource Energy
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,NEW YORK TIMES COMPANY,NEW YORK TIMES COMPANY,THE NEW YORK TIMES COMPANY,91.666667,...,United States,NaN,NaN,THE NEW YORK TIMES COMPANY,Compustat,7866.0,United States,7866.0,United States,The New York Times Company
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGÉTICAS NACIONAIS SGPS SA,96.202532,...,Portugal,285317.0,Portugal,NaN,NaN,285317.0,United States,285317.0,Portugal,"REN - Redes Energéticas Nacionais, SGPS, S.A."
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP INC,92.592593,...,United States,NaN,NaN,SELECTIVE INSURANCE GROUP INC,Compustat,17115.0,United States,17115.0,United States,"Selective Insurance Group, Inc."


In [95]:
drop_columns = ["new_gvkeys", "new_country", "final_gvkey", "final_country"]
rename_dict = {"manual_gvkey": "final_gvkey", "manual_country": "final_country", "manual_fullnames": "final_fullnames"}
df = df.drop(drop_columns, axis = 1)
df = df.rename(rename_dict, axis = 1)
df

,Keywords,Paragraph,Date,Title,Subtitle,Report,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_No_Punctuation_Compustat_Closest_Match,Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,...,Cleaned_No_Punctuation_Hassan_Closest_Match,Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity,Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey,Cleaned_No_Punctuation_Hassan_Closest_Match_Fullname,Cleaned_No_Punctuation_Hassan_Closest_Match_country,Manual_Match_Firm,Manual_Match_Type,final_gvkey,final_country,final_fullnames
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,TELIOS PHARMACEUTICALS,85.714286,...,IONIS PHARMACEUTICALS INC,84.444444,24040,Ionis Pharmaceuticals Inc,United States,IONIS PHARMACEUTICALS INC,Compustat,24040.0,United States,"Ionis Pharmaceuticals, Inc."
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA COMPANY,84.615385,...,AMERICANAS SA,68.571429,163677,Americanas SA,Brazil,AMERICAN ITALIAN PASTA COMPANY,Compustat,65570.0,United States,American Italian Pasta Company
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,GREENLIGHT COMMUNICATIONS,85.106383,...,INVISIO COMMUNICATIONS AB,80.851064,278126,Invisio Communications AB,Sweden,INSIGHT COMMUNICATIONS COMPANY LLC,Hassan,122096.0,United States,"Insight Communications Company, Inc."
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,KEYSPAN CORP.,KEYSPAN CORP,KEYERA CORP,78.260870,...,KEYSPAN CORP,100.000000,6799,Keyspan Corp,United States,NaN,NaN,6799.0,United States,KeySpan Corporation
4,irr,people per year. It has 300 retail stores inc...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,8710501,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INC,84.210526,...,SIMON PROPERTY GROUP INC,84.210526,29389,Simon Property Group Inc,United States,SIMON PROPERTY GROUP INC,Compustat,29389.0,United States,"Simon Property Group, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,100.000000,...,EVERSOURCE ENERGY,100.000000,7970,Eversource Energy,United States,NaN,NaN,7970.0,United States,Eversource Energy
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,NEW YORK TIMES COMPANY,NEW YORK TIMES COMPANY,THE NEW YORK TIMES COMPANY,91.666667,...,NEW YORK TIMES CO,87.179487,7866,New York Times Co,United States,THE NEW YORK TIMES COMPANY,Compustat,7866.0,United States,The New York Times Company
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGÉTICAS NACIONAIS SGPS SA,96.202532,...,REN REDES ENERGETICAS NACIONAIS SGPS SA,100.000000,285317,REN Redes Energeticas Nacionais SGPS SA,Portugal,NaN,NaN,285317.0,Portugal,"REN - Redes Energéticas Nacionais, SGPS, S.A."
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP INC,92.592593,...,SELECTIVE INSURANCE GROUP INC,92.592593,17115,Selective Insurance Group I

In [96]:
df.to_excel("manual_full_updated_conf_calls.xlsx", index = False)

In [97]:
matched = df[df["final_gvkey"].notnull()]
matched#.to_excel("manual_matched_updated_conf_calls.xlsx", index = False)

,Keywords,Paragraph,Date,Title,Subtitle,Report,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_No_Punctuation_Compustat_Closest_Match,Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,...,Cleaned_No_Punctuation_Hassan_Closest_Match,Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity,Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey,Cleaned_No_Punctuation_Hassan_Closest_Match_Fullname,Cleaned_No_Punctuation_Hassan_Closest_Match_country,Manual_Match_Firm,Manual_Match_Type,final_gvkey,final_country,final_fullnames
0,interest rate,MS. LYNNE PARSHALL: We ended the first quarter...,2002-07-03,ISIS PHARMACEUTICALS (ISIS) - ISIS PHARMACEUTI...,Â,8008277,ISIS PHARMACEUTICALS,ISIS PHARMACEUTICALS,TELIOS PHARMACEUTICALS,85.714286,...,IONIS PHARMACEUTICALS INC,84.444444,24040,Ionis Pharmaceuticals Inc,United States,IONIS PHARMACEUTICALS INC,Compustat,24040.0,United States,"Ionis Pharmaceuticals, Inc."
1,interest rate,"35%, we anticipate margins in the fourth quar...",2002-07-24,AMERICAN ITALIAN PASTA (PLB) - Q3 2002 FINANCI...,Â,8009744,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA,AMERICAN ITALIAN PASTA COMPANY,84.615385,...,AMERICANAS SA,68.571429,163677,Americanas SA,Brazil,AMERICAN ITALIAN PASTA COMPANY,Compustat,65570.0,United States,American Italian Pasta Company
2,interest rate,MR. DINNI JAIN: Sure. As of June 30th we had 1...,2002-07-24,INSIGHT COMMUNICATIONS (ICCI) - Q2 2002 FINANC...,Â,8009801,INSIGHT COMMUNICATIONS,INSIGHT COMMUNICATIONS,GREENLIGHT COMMUNICATIONS,85.106383,...,INVISIO COMMUNICATIONS AB,80.851064,278126,Invisio Communications AB,Sweden,INSIGHT COMMUNICATIONS COMPANY LLC,Hassan,122096.0,United States,"Insight Communications Company, Inc."
3,discount rate,MR. GERRY LUTERMAN: This is Gerry Luterman. Le...,2002-07-25,KEYSPAN CORP. (KSE) - Q2 2002 FINANCIAL RELEAS...,Â,8009965,KEYSPAN CORP.,KEYSPAN CORP,KEYERA CORP,78.260870,...,KEYSPAN CORP,100.000000,6799,Keyspan Corp,United States,NaN,NaN,6799.0,United States,KeySpan Corporation
4,irr,people per year. It has 300 retail stores inc...,NaN,SIMON PROPERTY GROUP INCORPORATED (SPG) - Q2 2...,NaN,8710501,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INCORPORATED,SIMON PROPERTY GROUP INC,84.210526,...,SIMON PROPERTY GROUP INC,84.210526,29389,Simon Property Group Inc,United States,SIMON PROPERTY GROUP INC,Compustat,29389.0,United States,"Simon Property Group, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,CAPM,b. Expects to capitalize projects ...,2020-02-20,EVERSOURCE ENERGY,ES.N - Event Brief of Eversource Energy confer...,68931059,EVERSOURCE ENERGY,EVERSOURCE ENERGY,EVERSOURCE ENERGY,100.000000,...,EVERSOURCE ENERGY,100.000000,7970,Eversource Energy,United States,NaN,NaN,7970.0,United States,Eversource Energy
111456,CAPM,Unidentified Hold on one minute. It's 13% at t...,2009-01-28,NEW YORK TIMES COMPANY (THE) - PRELIMINARY TRA...,"NYT conference call Final transcript, 28-Jan-0...",14370045,NEW YORK TIMES COMPANY,NEW YORK TIMES COMPANY,THE NEW YORK TIMES COMPANY,91.666667,...,NEW YORK TIMES CO,87.179487,7866,New York Times Co,United States,THE NEW YORK TIMES COMPANY,Compustat,7866.0,United States,The New York Times Company
111457,CAPM,João Caetano Carreira Faria Conceição - REN - ...,2019-05-06,REN REDES ENERGETICAS NACIONAIS SGPS SA,RENE.LS - Event Transcript of Ren Redes Energe...,67257248,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGETICAS NACIONAIS SGPS SA,REN REDES ENERGÉTICAS NACIONAIS SGPS SA,96.202532,...,REN REDES ENERGETICAS NACIONAIS SGPS SA,100.000000,285317,REN Redes Energeticas Nacionais SGPS SA,Portugal,NaN,NaN,285317.0,Portugal,"REN - Redes Energéticas Nacionais, SGPS, S.A."
111458,CAPM,_____________________________________________ ...,2003-08-06,SELECTIVE INSURANCE GROUP (SIGI) - Q2 2003 FIN...,,8906619,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP,SELECTIVE INSURANCE GROUP INC,92.592593,...,SELECTIVE INSURANCE GROUP INC,92.592593,17115,Selective Insurance Group I

In [98]:
unmatched = df[df["final_gvkey"].isnull()]
unmatched#.to_excel("manual_unmatched_updated_conf_calls.xlsx", index = False)

,Keywords,Paragraph,Date,Title,Subtitle,Report,Cleaned_Name,Cleaned_Name_No_Punctuations,Cleaned_No_Punctuation_Compustat_Closest_Match,Cleaned_No_Punctuation_Compustat_Closest_Match_Similarity,...,Cleaned_No_Punctuation_Hassan_Closest_Match,Cleaned_No_Punctuation_Hassan_Closest_Match_Similarity,Cleaned_No_Punctuation_Hassan_Closest_Match_gvkey,Cleaned_No_Punctuation_Hassan_Closest_Match_Fullname,Cleaned_No_Punctuation_Hassan_Closest_Match_country,Manual_Match_Firm,Manual_Match_Type,final_gvkey,final_country,final_fullnames
7,interest rate,in the second−quarter '02 compared to 7 cents...,2002-08-14,"HPSC, INC. (HDR) - Q2 2002 FINANCIAL RELEASE C...",Â,8712139,"HPSC, INC.",HPSC INC,HPVC INC,87.500000,...,HP INC,85.714286,5606,HP Inc,United States,NaN,NaN,NaN,NaN,NaN
25,discount rate,their customers in the European food service ...,2003-01-29,MCCORMICK & CO. (MKC) - Q4 2002 FINANCIAL RELE...,Â,8721747,MCCORMICK & CO.,MCCORMICK CO,MCCORMICK CAPITAL INC,70.588235,...,MCCORMICK COMPANY INC,74.285714,7146,McCormick & Company Inc,United States,NaN,NaN,NaN,NaN,NaN
27,return on assets,Accounts receivable and other current assets w...,NaN,AMERICAN STANDARD COS INC (ASD) - Q4 2002 FINA...,NaN,8721897,AMERICAN STANDARD COS INC,AMERICAN STANDARD COS INC,AMERICAN SNACKS INC,81.818182,...,AMERICAN SOFTWARE INC,73.913043,1562,American Software Inc,United States,NaN,NaN,NaN,NaN,NaN
32,return on assets,property of $21 million and we deduct proceeds...,NaN,AMERICAN STANDARD COS INC (ASD) - Q4 2002 FINA...,NaN,8722106,AMERICAN STANDARD COS INC,AMERICAN STANDARD COS INC,AMERICAN SNACKS INC,81.818182,...,AMERICAN SOFTWARE INC,73.913043,1562,American Software Inc,United States,NaN,NaN,NaN,NaN,NaN
41,cost of equity,the production goals that you have for '03 how...,2003-02-13,QUESTAR CORPORATION (STR) - Q4 2002 FINANCIAL ...,Â,8723383,QUESTAR CORPORATION,QUESTAR CORPORATION,QUALSTAR CORPORATION,92.307692,...,TELSTRA CORPORATION LTD,76.190476,210216,Telstra Corporation Ltd,Australia,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111387,OCC,efforts and therefore to end their attendance ...,2003-02-24,WEIGHT WATCHERS (WTW) - Q4 2002 FINANCIAL RELE...,,8724003,WEIGHT WATCHERS,WEIGHT WATCHERS,WITTCHEN SA,61.538462,...,WEIGHT WATCHERS INTERNATIONAL INC,62.500000,145308,Weight Watchers International Inc,United States,NaN,NaN,NaN,NaN,NaN
111436,OCC,"Unidentified Speaker Okay, thank you very much...",2005-11-08,WHEELING PITTSBURGH CORP,,10742931,WHEELING PITTSBURGH CORP,WHEELING PITTSBURGH CORP,WHEELING POWER COMPANY,65.217391,...,AMPCOPITTSBURGH CORP,68.181818,1613,Ampco-Pittsburgh Corp,United States,NaN,NaN,NaN,NaN,NaN
111437,OCC,A. Highlights: 1. Executing synergy a...,2018-01-29,WRK - EVENT BRIEF OF WRKCO INC CONFERENCE CALL...,WRK - Event Brief of WRKCO INC conference call...,64220431,WRK,WRK,W KRUK SA,50.000000,...,WRKCO INC,50.000000,29830,WRKCO Inc,United States,NaN,NaN,NaN,NaN,NaN
111438,OCC,Transportation costs are being negatively impa...,2018-01-29,WRK - EVENT TRANSCRIPT OF WRKCO INC CONFERENCE...,WRK - Event Transcript of WRKCO INC conference...,64220414,WRK,WRK,W KRUK SA,50.000000,...,WRKCO INC,50.000000,29830,WRKCO Inc,United States,NaN,NaN,NaN,NaN,NaN
